## Data Preparation

In [1]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from pymongo import MongoClient
import json, re
from re import sub
from decimal import Decimal
import dask.dataframe as dd
from dask.distributed import Client, progress
import dask.dataframe as dd
from dask.delayed import delayed

### Dask Client

In [2]:
client = Client(n_workers=4, threads_per_worker=1)
client

Client Scheduler: tcp://127.0.0.1:44481 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.63 GB


### Database Connection

In [3]:
DB_USER = 'anutibara'
DB_PASS = 'anutibara'
DB_HOST = 'scraping-cluster-7dtgt.gcp.mongodb.net'
DB_NAME = 'scraping_db'

try:
    client = MongoClient(f'mongodb+srv://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')
    print("Database connected successfully")
except Exception as e:
    print("Error to connect to database: ", e)
db = client.get_database(DB_NAME)
properties = db.properties

Database connected successfully


### Database Queries

In [4]:
properties.count_documents({})

1896

### Convert Mongo Collection to DataFrame

#### New Properties

In [5]:
new_properties_json = list(properties.find({ 'use': 'Nuevo' }))

In [6]:
df_new_properties = json_normalize(new_properties_json, record_path='offersType', 
                                meta=['_id', 
                                    'urlProperty',
                                    'scrapingDate', 
                                    'scrapingHour', 
                                    'modifyDate',
                                    'modifyHour', 
                                    'code', 
                                    'status', 
                                    'type', 
                                    'use', 
                                    'nameProject', 
                                    'description',
                                     'country',
                                     'department',
                                     'city',
                                     'sector',
                                     'neighborhood',
                                     'address',
                                     'latitude',
                                     'longitude',
                                     'idOwnerProperty',
                                     'nameOwnerProperty',
                                     'contractType',
                                     'financing',
                                     'schedule',
                                     'price',
                                     'squareMeters',
                                     'rooms',
                                     'bathrooms',
                                     'garages',
                                     'privateArea',
                                     'constructionArea',
                                     'squareMetersPrice',
                                     'stratum',
                                     'condition',
                                     'antiquity',
                                     'floor',
                                     'interiorFloors',
                                     'weather',
                                     'includesAdministration',
                                     'admonPrice',
                                     'interiorFeatures',
                                     'exteriorFeatures',
                                     'sectorFeatures'])
ddf_new_properties = dd.from_pandas(df_new_properties, npartitions=10)

In [7]:
ddf_new_properties = ddf_new_properties.loc[:,~ddf_new_properties.columns.duplicated(keep='first')]

In [8]:
ddf_new_properties = ddf_new_properties.dropna()

#### Old Properties

In [9]:
old_properties_json = list(properties.find({ 'use': 'Usado' }))

In [10]:
df_old_properties = pd.DataFrame(old_properties_json, 
                                    columns=['_id', 
                                    'urlProperty', 
                                    'scrapingDate', 
                                    'scrapingHour', 
                                    'modifyDate',
                                    'modifyHour', 
                                    'code', 
                                    'status', 
                                    'type', 
                                    'use', 
                                    'nameProject', 
                                    'description',
                                    'country',
                                     'department',
                                     'city',
                                     'sector',
                                     'neighborhood',
                                     'address',
                                     'latitude',
                                     'longitude',
                                     'idOwnerProperty',
                                     'nameOwnerProperty',
                                     'contractType',
                                     'financing',
                                     'schedule',
                                     'price',
                                     'squareMeters',
                                     'rooms',
                                     'bathrooms',
                                     'garages',
                                     'privateArea',
                                     'constructionArea',
                                     'squareMetersPrice',
                                     'stratum',
                                     'condition',
                                     'antiquity',
                                     'floor',
                                     'interiorFloors',
                                     'weather',
                                     'includesAdministration',
                                     'admonPrice',
                                     'interiorFeatures',
                                     'exteriorFeatures',
                                     'sectorFeatures'])
ddf_old_properties = dd.from_pandas(df_old_properties, npartitions=10)

In [11]:
ddf_old_properties = ddf_old_properties.loc[:,~ddf_old_properties.columns.duplicated(keep='first')]

In [12]:
ddf_old_properties = ddf_old_properties.dropna()

### Rename DataFrame Columns

#### New Properties

In [13]:
ddf_new_properties = ddf_new_properties.rename(columns={
                        "_id": "id_mongoose", 
                        "urlProperty":"id_property",
                        "scrapingDate": "scraping_date",
                        "scrapingHour": "scraping_hour",
                        "modifyDate": "modify_date",
                        "modifyHour": "modify_hour",
                        "status": "active",
                        "use": "new_property",
                        "nameProject": "name_project",
                        "offersType": "offers_type",
                        "idOwnerProperty": "id_owner_property",
                        "nameOwnerProperty": "name_owner_property",
                        "contractType": "contract_type_owner_property",
                        "financing": "financing_owner_property",
                        "schedule": "schedule_owner_property",
                        "bathrooms": "general_bathrooms",
                        "rooms": "general_rooms",
                        "price": "range_prices",
                        "squareMeters": "range_square_meters",
                        "constructionArea": "range_construction_area",
                        "squareMetersPrice": "square_meters_price",
                        "interiorFloors": "interior_floors",
                        "includesAdministration": "includes_administration",
                        "admonPrice": "admon_price",
                        "interiorFeatures": "interior_features",
                        "exteriorFeatures": "exterior_features",
                        "sectorFeatures": "sector_features",
                        "offerType": "offer_type",
                        "privateArea": "range_private_area",
                        "areaOfferType": "area",
                        "bathroomsOfferType": "bathrooms",
                        "priceOfferType": "price",
                        "privateAreaOfferType": "private_area",
                        "roomsOfferType": "rooms"
})

#### Old Properties

In [14]:
ddf_old_properties = ddf_old_properties.rename(columns={
                        "_id": "id_mongoose", 
                        "urlProperty":"id_property",
                        "scrapingDate": "scraping_date",
                        "scrapingHour": "scraping_hour",
                        "modifyDate": "modify_date",
                        "modifyHour": "modify_hour",
                        "status": "active",
                        "use": "new_property",
                        "nameProject": "name_project",
                        "offersType": "offers_type",
                        "idOwnerProperty": "id_owner_property",
                        "nameOwnerProperty": "name_owner_property",
                        "contractType": "contract_type_owner_property",
                        "financing": "financing_owner_property",
                        "schedule": "schedule_owner_property",
                        "squareMeters": "square_meters",
                        "privateArea": "private_area",
                        "constructionArea": "construction_area",
                        "squareMetersPrice": "square_meters_price",
                        "interiorFloors": "interior_floors",
                        "includesAdministration": "includes_administration",
                        "admonPrice": "admon_price",
                        "offerType": "offer_type",
                        "areaOfferType": "area",
                        "bathroomsOfferType": "bathrooms",
                        "priceOfferType": "price",
                        "privateAreaOfferType": "private_area",
                        "roomsOfferType": "rooms"
})

### Data Cleaning

#### New Properties

In [15]:
ddf_new_properties['id_mongoose'] = ddf_new_properties['id_mongoose'].astype(str)
ddf_new_properties['code'] = ddf_new_properties['code'].astype(int)
ddf_new_properties['active'] = ddf_new_properties['active'].apply(lambda status: True if (status == 'Active') else False, meta=('active', 'bool'))
ddf_new_properties['new_property'] = ddf_new_properties['new_property'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('new_property', 'bool'))
ddf_new_properties['includes_administration'] = ddf_new_properties['includes_administration'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('includes_administration', 'bool'))
ddf_new_properties['garages'] = ddf_new_properties['garages'].replace('', value = 0, regex = True).astype(int)
ddf_new_properties['stratum'] = ddf_new_properties['stratum'].replace('', value = 0, regex = True).astype(int)
ddf_new_properties['floor'] = ddf_new_properties['floor'].astype(int)
ddf_new_properties['area'] = ddf_new_properties['area'].astype(float)
ddf_new_properties['private_area'] = ddf_new_properties['private_area'].replace('', value = 0, regex = True)
ddf_new_properties['private_area'] = ddf_new_properties['private_area'].astype(float)
ddf_new_properties['rooms'] = ddf_new_properties['rooms'].astype(int)
ddf_new_properties['bathrooms'] = ddf_new_properties['bathrooms'].astype(int)
ddf_new_properties['price'] = ddf_new_properties['price'].apply(lambda price: Decimal(sub(r'[^\d,]', '', price)), meta=('price', 'int'))
ddf_new_properties['price'] = ddf_new_properties['price'].astype(int)

#### Old Properties

In [16]:
ddf_old_properties['id_mongoose'] = ddf_old_properties['id_mongoose'].astype(str)
ddf_old_properties['code'] = ddf_old_properties['code'].astype(int)
ddf_old_properties['active'] = ddf_old_properties['active'].apply(lambda status: True if (status == 'Active') else False, meta=('active', 'bool'))
ddf_old_properties['new_property'] = ddf_old_properties['new_property'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('new_property', 'bool'))
ddf_old_properties['includes_administration'] = ddf_old_properties['includes_administration'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('includes_administration', 'bool'))
ddf_old_properties['garages'] = ddf_old_properties['garages'].replace('', value = 0, regex = True)
ddf_old_properties['garages'] = ddf_old_properties['garages'].replace('Más de 10', value = 0, regex = True)
ddf_old_properties['garages'] = ddf_old_properties['garages'].astype(int)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].replace('', value = 0, regex = True)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].replace('Campestre', value = 0, regex = True)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].astype(int)
ddf_old_properties['price'] = ddf_old_properties['price'].apply(lambda price: Decimal(sub(r'[^\d,]', '', price)), meta=('price', 'int'))
ddf_old_properties['price'] = ddf_old_properties['price'].astype(int)
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].str[0:-3]
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].apply(lambda meters: Decimal(sub(r'[^\d,]', '', meters)), meta=('square_meters', 'float'))
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].astype(float)
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].str[0:-2]
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].replace('', value = '0', regex = True)
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].apply(lambda area: Decimal(sub(r'[^\d.]', '', area)), meta=('private_area', 'float'))
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].astype(float)
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].str[0:-3]
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].apply(lambda area: Decimal(sub(r'[^\d,]', '', area)), meta=('construction_area', 'float'))
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].astype(float)
ddf_old_properties['floor'] = ddf_old_properties['floor'].astype(int)

### Testing

#### New Properties

In [17]:
ddf_new_properties.describe().head()

,area,private_area,rooms,bathrooms,price,code,garages,stratum,floor
count,3234.000000,3234.000000,3234.000000,3234.000000,3.234000e+03,3.234000e+03,3234.000000,3234.000000,3234.0
mean,82.776645,62.961645,2.432900,2.297155,5.053441e+08,2.780311e+06,0.858998,5.372913,0.0
std,22.394122,33.317993,0.843075,0.791900,1.647036e+08,5.620691e+05,0.436398,0.921655,0.0
min,41.750000,0.000000,1.000000,1.000000,2.062000e+08,1.518663e+06,0.000000,4.000000,0.0
25%,76.400000,56.130000,3.000000,2.000000,3.956325e+08,2.565842e+06,1.000000,6.000000,0.0


#### Old Properties

In [18]:
ddf_old_properties.describe().head()

,code,latitude,longitude,id_owner_property,price,square_meters,rooms,bathrooms,garages,private_area,construction_area,square_meters_price,stratum,floor,interior_floors,admon_price
count,1.558000e+03,1558.000000,1558.000000,1558.000000,1.558000e+03,1558.000000,1558.000000,1558.000000,1558.000000,1558.000000,1558.000000,1.558000e+03,1558.000000,1558.000000,1558.000000,1.558000e+03
mean,4.678477e+06,5.206960,-62.429049,493242.691271,3.319741e+08,219.950578,2.299101,2.380616,0.909499,90.163863,219.950578,4.298656e+06,4.862644,2.407574,0.050064,1.626652e+05
std,5.708427e+05,2.832496,28.787702,482790.588951,6.253754e+08,1852.562147,1.726781,1.541399,1.262634,117.573236,1852.562147,8.867612e+07,1.267297,4.328825,0.295649,1.840057e+06
min,8.224820e+05,-6.165193,-75.797394,28812.000000,4.000000e+05,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.667000e+01,0.000000,0.000000,0.000000,0.000000e+00
25%,4.820658e+06,6.201599,-75.581118,210587.000000,2.750000e+06,82.000000,2.000000,2.000000,0.000000,0.000000,82.000000,3.000000e+04,5.000000,0.000000,0.000000,0.000000e+00
